### Imports

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import PyPDF2
import numpy as np
import re

#### Test Code

In [2]:
url = 'http://www.espn.com/college-football/standings/_/season/2016'

a = requests.get(url)
page = a.text
soup = BeautifulSoup(page,"html5lib")

In [3]:
print(soup.prettify)

<bound method Tag.prettify of <!DOCTYPE html>
<html class="no-icon-fonts"><head>
		<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<meta content="IE=edge,chrome=1" http-equiv="x-ua-compatible"/>
<meta content="initial-scale=1.0, maximum-scale=1.0, user-scalable=no" name="viewport"/>
<link href="http://www.espn.com/college-football/standings/_/season/2016" rel="canonical"/>
<title>NCAA FBS (Division I-A) Football Standings</title>
<meta content="Check out the 2016 NCAA FBS (Division I-A) College Football Standings on ESPN" name="description"/>
<meta content="" name="news_keywords"/>
<meta content="" name="keywords"/>
<meta content="116656161708917" property="fb:app_id"/>
<meta content="ESPN.com" property="og:site_name"/>
<meta content="http://www.espn.com/college-football/standings/_/season/2016" property="og:url"/>
<meta content="NCAA FBS (Division I-A) Football Standings" property="og:title"/>
<meta content="Check out the 2016 NCAA FBS (Division I-A) College Footb

In [4]:
#pdf_search = soup.find_all('td') #(style_ = "white-space:no-wrap;")
pdf_search = soup.find_all(class_ = '')
pdf_search

[<head>
 		<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
 <meta content="IE=edge,chrome=1" http-equiv="x-ua-compatible"/>
 <meta content="initial-scale=1.0, maximum-scale=1.0, user-scalable=no" name="viewport"/>
 <link href="http://www.espn.com/college-football/standings/_/season/2016" rel="canonical"/>
 <title>NCAA FBS (Division I-A) Football Standings</title>
 <meta content="Check out the 2016 NCAA FBS (Division I-A) College Football Standings on ESPN" name="description"/>
 <meta content="" name="news_keywords"/>
 <meta content="" name="keywords"/>
 <meta content="116656161708917" property="fb:app_id"/>
 <meta content="ESPN.com" property="og:site_name"/>
 <meta content="http://www.espn.com/college-football/standings/_/season/2016" property="og:url"/>
 <meta content="NCAA FBS (Division I-A) Football Standings" property="og:title"/>
 <meta content="Check out the 2016 NCAA FBS (Division I-A) College Football Standings on ESPN" property="og:description"/>
 <meta co

In [5]:
table = soup.find_all("td")
records = []
for item in table:
    #print(item.text)
    records.append(item.text)
    
print(np.array(records).reshape(-1, 12)[0])

['TempleTEM' '7-1' '267' '132' '10-4' '453' '258' '6-1' '4-2' 'L1' '1-1'
 '1-1']


### Get FBS records

In [6]:
years = ['2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016']
base = 'http://www.espn.com/college-football/standings/_/season/'
def get_fbs_records(years, base):
    """
    Scrapes the specified ESPN website and returns win-loss records for all fbs teams for the given years.
    
    INPUTS:
    years - a list of the years desired for scraping as strings
    base - a string of the desired website to scrape
    
    OUTPUTS:
    records_df - a dataframe consisting of all teams and their season stats from the given years
    """
    records_df = pd.DataFrame()
    for year in years:
        url = base + year
        get_page = requests.get(url)
        page = get_page.text
        soup = BeautifulSoup(page,"html5lib")
        table = soup.find_all("td")
        records = []
        for item in table:
        #print(item.text)
            records.append(item.text)
        records_array = np.array(records).reshape(-1, 12)
        #print(year, records_array.shape)
        df = pd.DataFrame(records_array)
        records_df = pd.concat([records_df, df], ignore_index=True)
    return records_df

fbs = get_fbs_records(years, base)


##### Changing column names and cleaning the dataframe

In [7]:
team_years = np.repeat(('2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016'), (120, 120, 120, 124, 126, 128, 128, 128))
fbs['YEARS'] = team_years
fbs.columns = ['TEAM', 'CONF', 'PF', 'PA', 'OVER', 'PF', 'PA', 'HOME', 'AWAY', 'STRK', 'AP', 'USA', 'YEARS']
fbs.head()

,TEAM,CONF,PF,PA,OVER,PF,PA,HOME,AWAY,STRK,AP,USA,YEARS
0,ClemsonCLEM,6-2,268,169,9-5,436,286,7-2,2-3,W1,1-3,1-3,2009
1,Boston CollegeBC,5-3,174,196,8-5,322,257,6-1,2-4,L1,0-3,0-2,2009
2,Florida StateFSU,4-4,268,278,7-6,391,390,4-3,3-3,W1,2-4,2-3,2009
3,Wake ForestWAKE,3-5,226,254,5-7,316,315,4-3,1-4,-,0-3,0-2,2009
4,NC StateNCST,2-6,213,315,5-7,364,374,5-3,0-4,-,1-2,1-1,2009


In [8]:
fbs_winning_perc_df = fbs[['TEAM', 'OVER', 'YEARS']]
fbs_winning_perc_df.head()

,TEAM,OVER,YEARS
0,ClemsonCLEM,9-5,2009
1,Boston CollegeBC,8-5,2009
2,Florida StateFSU,7-6,2009
3,Wake ForestWAKE,5-7,2009
4,NC StateNCST,5-7,2009


### Convert overall recored in the form of wins-losses to a win percentage

In [9]:
win_percent = []
for item in fbs_winning_perc_df['OVER'].values:
    win, loss = item.split('-')
    win_percent.append(int(win)/(int(win)+int(loss)))

fbs_winning_perc_df['WIN_PERCENT'] = np.array(win_percent)
fbs_winning_perc_df.head()

/Users/murdock/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,TEAM,OVER,YEARS,WIN_PERCENT
0,ClemsonCLEM,9-5,2009,0.642857
1,Boston CollegeBC,8-5,2009,0.615385
2,Florida StateFSU,7-6,2009,0.538462
3,Wake ForestWAKE,5-7,2009,0.416667
4,NC StateNCST,5-7,2009,0.416667


In [10]:
fbs_winning_perc_df = fbs_winning_perc_df[['TEAM', 'WIN_PERCENT', 'YEARS']]
fbs_winning_perc_df.head()

,TEAM,WIN_PERCENT,YEARS
0,ClemsonCLEM,0.642857,2009
1,Boston CollegeBC,0.615385,2009
2,Florida StateFSU,0.538462,2009
3,Wake ForestWAKE,0.416667,2009
4,NC StateNCST,0.416667,2009


### Get FCS records

In [11]:
years2 = ['2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016']
base2 = 'http://www.espn.com/college-football/standings/_/view/fcs/season/'
def get_fcs_records(years, base):
    """
    Scrapes the specified ESPN website and returns win-loss records for all fcs teams for the given years.
    
    INPUTS:
    years - a list of the years desired for scraping as strings
    base - a string of the desired website to scrape
    
    OUTPUTS:
    records_df - a dataframe consisting of all teams and their season stats from the given years
    """
    fcs_records_df = pd.DataFrame()
    for year in years:
        url = base + year
        get_page = requests.get(url)
        page = get_page.text
        soup = BeautifulSoup(page,"html5lib")
        table = soup.find_all("td")
        records = []
        for item in table:
            records.append(item.text)
        records_array = np.array(records).reshape(-1, 8)
        print(records_array)
        df2 = pd.DataFrame(records_array)
        fcs_records_df = pd.concat([fcs_records_df, df2], ignore_index=True)
    return fcs_records_df


fcs = get_fcs_records(years2, base2)

[['MontanaMONT' '8-0' '290' '164' '14-1' '537' '285' 'L1']
 ['Eastern WashingtonEWU' '6-2' '288' '184' '8-4' '404' '329' '-']
 ['Weber StateWEB' '6-2' '246' '155' '7-5' '338' '260' '-']
 ['Montana StateMTST' '5-3' '200' '156' '7-4' '257' '244' '-']
 ['Northern ArizonaNAU' '4-4' '254' '216' '5-6' '327' '327' '-']
 ['Sacramento StateSAC' '4-4' '221' '267' '5-6' '274' '371' '-']
 ['Northern ColoradoUNCO' '1-7' '147' '210' '3-8' '202' '288' '-']
 ['Portland StatePRST' '1-7' '160' '294' '2-9' '232' '372' '-']
 ['Idaho StateIDST' '1-7' '117' '277' '1-10' '142' '424' '-']
 ['LibertyLIB' '5-1' '271' '123' '8-3' '400' '210' '-']
 ['Stony BrookSTON' '5-1' '182' '121' '6-5' '267' '256' '-']
 ['Charleston SouthernCHSO' '4-2' '170' '150' '6-5' '268' '323' '-']
 ['Gardner-WebbWEBB' '3-3' '138' '158' '6-5' '305' '277' '-']
 ['Coastal CarolinaCCAR' '3-3' '133' '168' '5-6' '205' '280' '-']
 ['VMIVMI' '1-5' '115' '179' '2-9' '225' '338' '-']
 ['Presbyterian CollegePRE' '0-6' '136' '246' '0-11' '214' '44

[['MontanaMONT' '7-1' '272' '152' '11-3' '474' '279' 'L1']
 ['Montana StateMTST' '7-1' '285' '196' '10-3' '415' '318' '-']
 ['Portland StatePRST' '5-3' '261' '243' '7-4' '362' '308' '-']
 ['Eastern WashingtonEWU' '5-3' '259' '221' '6-5' '356' '332' '-']
 ['Weber StateWEB' '5-3' '270' '230' '5-6' '347' '354' '-']
 ['Northern ArizonaNAU' '3-5' '217' '213' '4-7' '309' '294' '-']
 ['Sacramento StateSAC' '3-5' '199' '215' '4-7' '261' '301' '-']
 ['Idaho StateIDST' '1-7' '136' '277' '2-9' '204' '404' '-']
 ['Northern ColoradoUNCO' '0-8' '148' '300' '0-11' '207' '382' '-']
 ['Stony BrookSTON' '6-0' '293' '128' '9-4' '494' '303' '-']
 ['LibertyLIB' '5-1' '231' '138' '7-4' '395' '242' '-']
 ['Coastal CarolinaCCAR' '3-3' '145' '187' '7-4' '271' '307' '-']
 ['Presbyterian CollegePRE' '3-3' '163' '118' '4-7' '276' '347' '-']
 ['Gardner-WebbWEBB' '2-4' '109' '201' '4-7' '199' '350' '-']
 ['VMIVMI' '2-4' '113' '192' '2-9' '187' '351' '-']
 ['Charleston SouthernCHSO' '0-6' '123' '213' '0-11' '186' '4

[['Eastern WashingtonEWU' '8-0' '338' ..., '592' '424' 'L1']
 ['Northern ArizonaNAU' '7-1' '228' ..., '278' '264' '-']
 ['MontanaMONT' '6-2' '292' ..., '490' '298' '-']
 ..., 
 ['Arkansas-Pine BluffARPB' '2-7' '285' ..., '310' '446' '-']
 ['Texas SouthernTXSO' '2-7' '150' ..., '173' '329' '-']
 ['Grambling StateGRAM' '1-8' '156' ..., '216' '425' '-']]
[['Eastern WashingtonEWU' '7-1' '334' '206' '11-3' '618' '439' 'L1']
 ['MontanaMONT' '6-2' '245' '170' '9-5' '415' '294' 'L1']
 ['Montana StateMTST' '6-2' '295' '254' '8-5' '496' '433' 'L2']
 ['Idaho StateIDST' '6-2' '345' '233' '8-4' '484' '377' 'W1']
 ['Northern ArizonaNAU' '5-3' '198' '198' '7-5' '315' '308' 'L2']
 ['Cal PolyCP' '5-3' '302' '254' '7-5' '406' '343' 'W1']
 ['Sacramento StateSAC' '4-4' '294' '320' '7-5' '458' '433' 'W3']
 ['Southern UtahSUU' '3-5' '223' '276' '3-9' '278' '427' 'W1']
 ['North DakotaUND' '3-5' '152' '218' '5-7' '191' '314' 'W2']
 ['Portland StatePRST' '2-6' '196' '240' '3-9' '290' '408' 'L4']
 ['Weber State

[['Eastern WashingtonEWU' '8-0' '363' '195' '12-2' '593' '371' 'L1']
 ['North DakotaUND' '8-0' '249' '153' '9-3' '355' '264' 'L1']
 ['Weber StateWEB' '6-2' '229' '211' '7-5' '312' '360' 'L1']
 ['Cal PolyCP' '5-3' '292' '282' '7-5' '416' '394' 'L1']
 ['Southern UtahSUU' '5-3' '286' '228' '6-5' '321' '312' 'W1']
 ['Northern ColoradoUNCO' '4-4' '245' '272' '6-5' '377' '398' 'L1']
 ['Northern ArizonaNAU' '4-4' '255' '232' '5-6' '361' '313' 'L2']
 ['MontanaMONT' '3-5' '306' '245' '6-5' '434' '297' 'L2']
 ['Sacramento StateSAC' '2-6' '227' '357' '2-9' '267' '440' 'L1']
 ['Montana StateMTST' '2-6' '183' '232' '4-7' '282' '276' 'W2']
 ['Portland StatePRST' '2-6' '284' '296' '3-8' '365' '429' 'L2']
 ['UC DavisUCD' '2-6' '208' '275' '3-8' '310' '408' 'W1']
 ['Idaho StateIDST' '1-7' '206' '355' '2-9' '267' '451' 'L7']
 ['Charleston SouthernCHSO' '4-1' '159' '60' '7-4' '363' '244' 'L1']
 ['LibertyLIB' '4-1' '153' '117' '6-5' '302' '286' 'L2']
 ['Kennesaw StateKENN' '3-2' '169' '130' '8-3' '428' '2

##### Change column names and clean dataframe

In [12]:
team_years2 = np.repeat(('2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016'), (125, 124, 125, 122, 126, 124, 125, 125))
fcs['YEARS'] = team_years2
fcs.columns = ['TEAM', 'CONF', 'PF', 'PA', 'OVER', 'PF', 'PA', 'STRK', 'YEARS']
fcs.head()

,TEAM,CONF,PF,PA,OVER,PF,PA,STRK,YEARS
0,MontanaMONT,8-0,290,164,14-1,537,285,L1,2009
1,Eastern WashingtonEWU,6-2,288,184,8-4,404,329,-,2009
2,Weber StateWEB,6-2,246,155,7-5,338,260,-,2009
3,Montana StateMTST,5-3,200,156,7-4,257,244,-,2009
4,Northern ArizonaNAU,4-4,254,216,5-6,327,327,-,2009


In [13]:
fcs_winning_perc_df = fcs[['TEAM', 'OVER', 'YEARS']]
fcs_winning_perc_df.head()

,TEAM,OVER,YEARS
0,MontanaMONT,14-1,2009
1,Eastern WashingtonEWU,8-4,2009
2,Weber StateWEB,7-5,2009
3,Montana StateMTST,7-4,2009
4,Northern ArizonaNAU,5-6,2009


### Convert overall recored in the form of wins-losses to a win percentage

In [14]:
win_percent = []
for item in fcs_winning_perc_df['OVER'].values:
    try: 
        win, loss = item.split('-')
        win_percent.append(int(win)/(int(win)+int(loss)))
    except ValueError:
        win, loss, ties = item.split('-')
        win_percent.append(int(win)/(int(win)+int(loss) + int(ties)))

fcs_winning_perc_df['WIN_PERCENT'] = np.array(win_percent)
fcs_winning_perc_df.head()

/Users/murdock/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,TEAM,OVER,YEARS,WIN_PERCENT
0,MontanaMONT,14-1,2009,0.933333
1,Eastern WashingtonEWU,8-4,2009,0.666667
2,Weber StateWEB,7-5,2009,0.583333
3,Montana StateMTST,7-4,2009,0.636364
4,Northern ArizonaNAU,5-6,2009,0.454545


In [15]:
fcs_winning_perc_df = fcs_winning_perc_df[['TEAM', 'WIN_PERCENT', 'YEARS']]
fcs_winning_perc_df.tail()

,TEAM,WIN_PERCENT,YEARS
991,GramblingGRAM,0.923077,2016
992,SouthernSOU,0.727273,2016
993,Prairie ViewPVAM,0.636364,2016
994,Texas SouthernTXSO,0.363636,2016
995,Arkansas-Pine BluffUAPB,0.090909,2016


### Merge FCS and FBS Teams

In [16]:
win_percentages_df = pd.concat([fbs_winning_perc_df, fcs_winning_perc_df], ignore_index=True)
win_percentages_df.head()

,TEAM,WIN_PERCENT,YEARS
0,ClemsonCLEM,0.642857,2009
1,Boston CollegeBC,0.615385,2009
2,Florida StateFSU,0.538462,2009
3,Wake ForestWAKE,0.416667,2009
4,NC StateNCST,0.416667,2009


In [17]:
win_percentages_df.tail()

,TEAM,WIN_PERCENT,YEARS
1985,GramblingGRAM,0.923077,2016
1986,SouthernSOU,0.727273,2016
1987,Prairie ViewPVAM,0.636364,2016
1988,Texas SouthernTXSO,0.363636,2016
1989,Arkansas-Pine BluffUAPB,0.090909,2016


In [ ]:
win_perc_path = '/Users/murdock/Documents/PDF_files/'
win_percentages_df.to_pickle(win_perc_path + 'win_perc_df.pkl')

### Use regex to fix team names before saving

In [18]:
win_percentages_df['TEAM'] = win_percentages_df['TEAM'].replace('San José StateSJSU', 'San Jose StateSJSU')

In [19]:
win_percentages_df['TEAM'] = win_percentages_df['TEAM'].replace('Miami (OH)M-OH', 'Miami OhM-OH')

In [20]:
win_percentages_df[win_percentages_df['TEAM'].str.contains('Miami')]

,TEAM,WIN_PERCENT,YEARS
8,MiamiMIAMI,0.692308,2009
64,Miami OhM-OH,0.083333,2009
127,MiamiMIAMI,0.538462,2010
178,Miami OhM-OH,0.714286,2010
249,MiamiMIAMI,0.500000,2011
302,Miami OhM-OH,0.333333,2011
368,MiamiMIAMI,0.583333,2012
422,Miami OhM-OH,0.333333,2012
503,MiamiMIAMI,0.692308,2013
557,Miami OhM-OH,0.000000,2013


In [21]:
win_percentages_df.head()

,TEAM,WIN_PERCENT,YEARS
0,ClemsonCLEM,0.642857,2009
1,Boston CollegeBC,0.615385,2009
2,Florida StateFSU,0.538462,2009
3,Wake ForestWAKE,0.416667,2009
4,NC StateNCST,0.416667,2009


In [22]:
teams = []
for team in win_percentages_df['TEAM'].values:
    team_regex = re.compile(r'^[A-Z]+([a-z]+)?(\s[A-Z])?([a-z]+)?(\W[A-Z])?(\s[A-Z])?([a-z]+)?')
    cleaned_team = team_regex.search(team).group()
    teams.append(cleaned_team)
    
win_percentages_df['TEAMS'] = teams

In [23]:
win_percentages_df.head()

,TEAM,WIN_PERCENT,YEARS,TEAMS
0,ClemsonCLEM,0.642857,2009,Clemson
1,Boston CollegeBC,0.615385,2009,Boston College
2,Florida StateFSU,0.538462,2009,Florida State
3,Wake ForestWAKE,0.416667,2009,Wake Forest
4,NC StateNCST,0.416667,2009,NC State


In [24]:
win_percent_df = win_percentages_df[['TEAMS', 'WIN_PERCENT', 'YEARS']]

In [25]:
win_percent_df.head()

,TEAMS,WIN_PERCENT,YEARS
0,Clemson,0.642857,2009
1,Boston College,0.615385,2009
2,Florida State,0.538462,2009
3,Wake Forest,0.416667,2009
4,NC State,0.416667,2009


In [26]:
win_percent_df[win_percent_df['TEAMS'] == 'Mc']

,TEAMS,WIN_PERCENT,YEARS
1102,Mc,0.750000,2009
1225,Mc,0.545455,2010
1353,Mc,0.545455,2011
1475,Mc,0.636364,2012
1599,Mc,0.769231,2013
1724,Mc,0.545455,2014
1844,Mc,0.909091,2015
1972,Mc,0.545455,2016


In [ ]:
win_perc_path = '/Users/murdock/Documents/PDF_files/'
win_percent_df.to_pickle(win_perc_path + 'win_perc_df.pkl')

### Fix regex and pickle final file

In [27]:
win_percent_df['TEAMS'] = win_percent_df['TEAMS'].replace('Mc', 'McNeese State')

/Users/murdock/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [28]:
win_percent_df[win_percent_df['TEAMS'] == 'McNeese State']

,TEAMS,WIN_PERCENT,YEARS
1102,McNeese State,0.750000,2009
1225,McNeese State,0.545455,2010
1353,McNeese State,0.545455,2011
1475,McNeese State,0.636364,2012
1599,McNeese State,0.769231,2013
1724,McNeese State,0.545455,2014
1844,McNeese State,0.909091,2015
1972,McNeese State,0.545455,2016


In [29]:
win_perc_path = '/Users/murdock/Documents/PDF_files/'
win_percent_df.to_pickle(win_perc_path + 'win_perc_df.pkl')